In [21]:
import tensorflow

from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import tensorflow as tf
import random
log_files_path = 'Data/Wavelet/Merged/'

#iloc makes sure we exclude our dependent variables/output
df=pd.read_csv(log_files_path + 'denoised2level2merged.csv')
d = df.values
ntotal=len(df)

d = normalize(d, axis=0, norm='l2')

resultu = []
np.random.rand(4)
# Return 100 results (for instance)
for i in range(ntotal):
    
    res = random.random()
    if res < 0.1:
        resultu.append(1)
    elif res < 0.2 and res>=0.1:
        resultu.append(2)
    elif res < 0.3 and res>=0.2:
        resultu.append(3)
    elif res < 0.4 and res>=0.3:
        resultu.append(4)
    elif res < 0.5 and res>=0.4:
        resultu.append(5)
    elif res < 0.6 and res>=0.5:
        resultu.append(6)
    elif res < 0.7 and res>=0.6:
        resultu.append(8)
    elif res < 0.8 and res>=0.7:
        resultu.append(8)
    else:
        resultu.append(9)
resultu=np.array(resultu)
trainset=[]
for i in range(1,7):
    toinsert=d[resultu==i].astype(np.float32)
    trainset.append(toinsert)
validationset=d[resultu==8].astype(np.float32)
testset=d[resultu==9].astype(np.float32)


#x = data

# Following Hinton-Salakhutdinov Architecture

n_code_1 = 17
n_code_2 = 13
n_code_3 = 9

# Parameters
learning_rate = 0.005

#batch_size = 7
display_step = 1

total_batch = 6
training_epochs = 5000

In [22]:
#we store the variables here
log_files_path = 'Output/WT3layer17-13-9/'
input_size = df.shape[1]
def layer_batch_normalization(x, n_out, phase_train):
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - n_out: integer, depth of input maps - number of sample in the batch 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - batch-normalized maps   
    
    
    """

    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)

    #tf.nn.moment: https://www.tensorflow.org/api_docs/python/tf/nn/moments
    #calculate mean and variance of x
    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
    
    #tf.train.ExponentialMovingAverage:
    #https://www.tensorflow.org/api_docs/python/tf/train/ExponentialMovingAverage
    #Maintains moving averages of variables by employing an exponential decay.
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
        
    #tf.cond: https://www.tensorflow.org/api_docs/python/tf/cond
    #Return true_fn() if the predicate pred is true else false_fn()
    mean, var = tf.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))
    
    
    #Here, we changed the shape of x into [[[x1]],[[x2] ],....]

    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    
    return tf.reshape(normed, [-1, n_out])

def layer(x, weight_shape, bias_shape, phase_train):
    
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - weight_shape: shape the the weight maxtrix
        - bias_shape: shape of the bias vector
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector of the layer after the matrix multiplication and non linear transformation
    """
    
    #initialize weights
    weight_init = tf.random_normal_initializer(stddev=(1.0/weight_shape[0])**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)

    logits = tf.matmul(x, W) + b
    
    #apply the non-linear function after the batch normalization
    return tf.nn.sigmoid(layer_batch_normalization(logits, weight_shape[1], phase_train))
def encoder(x, n_code_1, n_code_2, n_code_3, phase_train):
    
    """
    Defines the network encoder part
    input:
        - x: input vector of the encoder
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder)
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reduced dimension
    """
    
    with tf.variable_scope("encoder1", reuse=tf.AUTO_REUSE):
        encoder1 = layer(x, [input_size, n_code_1], [n_code_1], phase_train)
 
    with tf.variable_scope("encoder2", reuse=tf.AUTO_REUSE):
        encoder2 = layer(encoder1, [n_code_1, n_code_2], [n_code_2], phase_train)
    
    with tf.variable_scope("encoder3", reuse=tf.AUTO_REUSE):
        output = layer(encoder2, [n_code_2, n_code_3], [n_code_3], phase_train)

    
    return output


def decoder(x, n_code_1, n_code_2, n_code_3, phase_train):
    """
    Defines the network encoder part
    input:
        - x: input vector of the decoder - reduced dimension vector
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder) 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reconstructed dimension of the initial vector
    """
    

    with tf.variable_scope("decoder1", reuse=tf.AUTO_REUSE):
        decoder1 = layer(x, [n_code_3, n_code_2], [n_code_2], phase_train)
        
    with tf.variable_scope("decoder2", reuse=tf.AUTO_REUSE):
        decoder2 = layer(decoder1, [n_code_2, n_code_1], [n_code_1], phase_train)
    
    with tf.variable_scope("decoder3", reuse=tf.AUTO_REUSE):
        output = layer(decoder2, [n_code_1, input_size], [input_size], phase_train)
        
    return output


def loss(output, x):
    """
    Compute the loss of the auto-encoder
    
    intput:
        - output: the output of the decoder
        - x: true value of the sample batch - this is the input of the encoder
        
        the two have the same shape (batch_size * num_of_classes)
    output:
        - loss: loss of the corresponding batch (scalar tensor)
    
    """
    
    with tf.variable_scope("training", reuse=tf.AUTO_REUSE):
        
        l2 = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x)), 1))
        train_loss = tf.reduce_mean(l2)
        train_summary_op = tf.summary.scalar("train_cost", train_loss)
        return train_loss, train_summary_op
    
    
def training(cost, global_step):
    """
    defines the necessary elements to train the network
    
    intput:
        - cost: the cost is the loss of the corresponding batch
        - global_step: number of batch seen so far, it is incremented by one 
        each time the .minimize() function is called
    """
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op
def evaluate(output, x):
    """
    evaluates the accuracy on the validation set 
    input:
        -output: prediction vector of the network for the validation set
        -x: true value for the validation set
    output:
        - val_loss: loss of the autoencoder
        - in_image_op: input image 
        - out_image_op:reconstructed image 
        - val_summary_op: summary of the loss
    """
    
    with tf.variable_scope("validation"):
        
        #in_image_op = image_summary("input_image", x)
        
        #out_image_op = image_summary("output_image", output)
        
        l2_norm = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x, name="val_diff")), 1))
        
        val_loss = tf.reduce_mean(l2_norm)
        
        val_summary_op = tf.summary.scalar("val_cost", val_loss)
        
        #return val_loss, in_image_op, out_image_op, val_summary_op
        return val_loss,  val_summary_op
"""
def image_summary(label, tensor):
    #tf.summary.image: https://www.tensorflow.org/api_docs/python/tf/summary/image
    #Outputs a Summary protocol buffer with images.

    tensor_reshaped = tf.reshape(tensor, [-1, 19, 1, 1])
    return tf.summary.image(label, tensor_reshaped)
"""
if __name__ == '__main__':
    print('we begin')
    
    with tf.Graph().as_default():

        with tf.variable_scope("autoencoder_model"):

            #the input variables are first define as placeholder 
            # a placeholder is a variable/data which will be assigned later 
            # image vector & label, phase_train is a boolean 
            x = tf.placeholder("float", [None, input_size])
            
            phase_train = tf.placeholder(tf.bool)
            
            #define the encoder 
            code = encoder(x, n_code_1, n_code_2, n_code_3, phase_train)
            
            #define the decoder
            output = decoder(code, n_code_1, n_code_2, n_code_3, phase_train)
            
            #compute the loss 
            cost, train_summary_op = loss(output, x)

            #initialize the value of the global_step variable 
            # recall: it is incremented by one each time the .minimise() is called
            global_step = tf.Variable(0, name='global_step', trainable=False)

            train_op = training(cost, global_step)

            #evaluate the accuracy of the network (done on a validation set)
            #eval_op, in_image_op, out_image_op, val_summary_op = evaluate(output, x)
            eval_op, val_summary_op = evaluate(output, x)

            summary_op = tf.summary.merge_all()

            #save and restore variables to and from checkpoints.
            #saver = tf.train.Saver(max_to_keep=200)
            saver = tf.train.Saver()

            #defines a session
            sess = tf.Session()

            # summary writer
            #https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter
            #train_writer = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #val_writer   = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #initialization of the variables
            init_op = tf.global_variables_initializer()

            sess.run(init_op)
            currentmin=10000000000
            currentmin_index=-1
            error_path=[]
            togive2=[]
            # Training cycle
            for epoch in range(training_epochs):

                avg_cost = 0.
                #total_batch = int(numberofsamples/batch_size)
                
                # Loop over all batches
                for i in range(total_batch):
                    
                    minibatch_x= trainset[i]
                    
                    # Fit training using batch data
                    #the training is done using the training dataset
                    _, new_cost, train_summary = sess.run([train_op, cost, train_summary_op], feed_dict={x: minibatch_x, phase_train: True})
                    
                    #train_writer.add_summary(train_summary, sess.run(global_step))
                    
                    # Compute average loss
                    avg_cost += new_cost/total_batch
                    
                
                # Display logs per epoch step
                if epoch % display_step == 0:
                    print("Epoch:", '%04d' % (epoch+1), "cost =", "{:0.9f}".format(avg_cost))

                    #the accuracy is evaluated using the validation dataset
                    #train_writer.add_summary(train_summary, sess.run(global_step))

                    #validation_loss, in_image, out_image, val_summary = sess.run([eval_op, in_image_op, out_image_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    validation_loss,  val_summary = sess.run([eval_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    #val_writer.add_summary(in_image, sess.run(global_step))
                    #val_writer.add_summary(out_image, sess.run(global_step))
                    #val_writer.add_summary(val_summary, sess.run(global_step))
                    print("Validation Loss:", validation_loss)
                    error_path.append(validation_loss)
                    
                    if validation_loss<currentmin:
                        currentmin=validation_loss
                        currentmin_index=epoch
                        saver.save(sess, log_files_path + 'tmp/modelY.ckpt')
                        togive8=[]
                        for i in range(len(d)):
                            any_image = d[i].reshape(-1,input_size)
                            output_any_image = sess.run(code,feed_dict={x:any_image,phase_train: False})
                            togive8.append(output_any_image)
                            
                        togive2=togive8
                    #save to use later
                    #https://www.tensorflow.org/api_docs/python/tf/train/Saver
                    #saver.save(sess, log_files_path+'model-checkpoint', global_step=global_step)
                    #saver.save(sess, 'C:/Users/yy2895/Desktop/st19-14-19/tmp/model.ckpt')
                    

            print("Optimization Finished!")

            test_loss = sess.run(eval_op, feed_dict={x: testset, phase_train: False})
            #nps_loss = sess.run(eval_op, feed_dict={x: mnist.validation.images, phase_train: False})
            print("Test Loss:", test_loss)
            

we begin
Epoch: 0001 cost = 2.527219971
Validation Loss: 2.9435747
Epoch: 0002 cost = 2.507289290
Validation Loss: 2.8085456
Epoch: 0003 cost = 2.487578829
Validation Loss: 2.5626943
Epoch: 0004 cost = 2.467819254
Validation Loss: 2.468191
Epoch: 0005 cost = 2.448202133
Validation Loss: 2.433617
Epoch: 0006 cost = 2.429149270
Validation Loss: 2.4141414
Epoch: 0007 cost = 2.410219431
Validation Loss: 2.5741954
Epoch: 0008 cost = 2.390829007
Validation Loss: 2.6386802
Epoch: 0009 cost = 2.370292902
Validation Loss: 2.762599
Epoch: 0010 cost = 2.347201228
Validation Loss: 2.988164
Epoch: 0011 cost = 2.320577025
Validation Loss: 3.0578153
Epoch: 0012 cost = 2.292606672
Validation Loss: 2.882964
Epoch: 0013 cost = 2.262795170
Validation Loss: 2.5791914
Epoch: 0014 cost = 2.234551549
Validation Loss: 2.173226
Epoch: 0015 cost = 2.207529227
Validation Loss: 1.9738038
Epoch: 0016 cost = 2.184602141
Validation Loss: 1.686661
Epoch: 0017 cost = 2.166698297
Validation Loss: 1.8205342
Epoch: 0018 

Epoch: 0150 cost = 1.195475996
Validation Loss: 1.1005594
Epoch: 0151 cost = 1.190625787
Validation Loss: 1.250888
Epoch: 0152 cost = 1.185884972
Validation Loss: 1.3378079
Epoch: 0153 cost = 1.181129893
Validation Loss: 1.3730454
Epoch: 0154 cost = 1.176332474
Validation Loss: 1.3291342
Epoch: 0155 cost = 1.171589454
Validation Loss: 1.322058
Epoch: 0156 cost = 1.166913708
Validation Loss: 1.1768347
Epoch: 0157 cost = 1.162294666
Validation Loss: 1.3326257
Epoch: 0158 cost = 1.157652179
Validation Loss: 1.355733
Epoch: 0159 cost = 1.153001249
Validation Loss: 1.1946609
Epoch: 0160 cost = 1.148397247
Validation Loss: 1.1063185
Epoch: 0161 cost = 1.143863241
Validation Loss: 1.0665861
Epoch: 0162 cost = 1.139379561
Validation Loss: 1.101203
Epoch: 0163 cost = 1.134867410
Validation Loss: 0.90903616
Epoch: 0164 cost = 1.130339622
Validation Loss: 0.9352113
Epoch: 0165 cost = 1.125865440
Validation Loss: 0.9212649
Epoch: 0166 cost = 1.121451080
Validation Loss: 1.0406126
Epoch: 0167 cost 

Epoch: 0293 cost = 0.723701070
Validation Loss: 1.067554
Epoch: 0294 cost = 0.721428643
Validation Loss: 0.9466051
Epoch: 0295 cost = 0.719359517
Validation Loss: 0.9288083
Epoch: 0296 cost = 0.717237453
Validation Loss: 0.7457016
Epoch: 0297 cost = 0.715329876
Validation Loss: 0.53697664
Epoch: 0298 cost = 0.712957750
Validation Loss: 0.5461394
Epoch: 0299 cost = 0.711029351
Validation Loss: 0.6134861
Epoch: 0300 cost = 0.708833297
Validation Loss: 0.71266365
Epoch: 0301 cost = 0.707092057
Validation Loss: 0.6923503
Epoch: 0302 cost = 0.704687546
Validation Loss: 0.7843135
Epoch: 0303 cost = 0.702849140
Validation Loss: 0.5996103
Epoch: 0304 cost = 0.700638831
Validation Loss: 0.5500876
Epoch: 0305 cost = 0.698970050
Validation Loss: 0.577941
Epoch: 0306 cost = 0.696622024
Validation Loss: 0.6654355
Epoch: 0307 cost = 0.694807112
Validation Loss: 0.7151561
Epoch: 0308 cost = 0.692643007
Validation Loss: 0.7103824
Epoch: 0309 cost = 0.690970878
Validation Loss: 0.7076679
Epoch: 0310 co

Epoch: 0444 cost = 0.499911174
Validation Loss: 0.50242954
Epoch: 0445 cost = 0.498900627
Validation Loss: 0.54132974
Epoch: 0446 cost = 0.497893497
Validation Loss: 0.49982858
Epoch: 0447 cost = 0.496997242
Validation Loss: 0.47103754
Epoch: 0448 cost = 0.495986079
Validation Loss: 0.40297282
Epoch: 0449 cost = 0.494994422
Validation Loss: 0.40172085
Epoch: 0450 cost = 0.494006117
Validation Loss: 0.51101154
Epoch: 0451 cost = 0.493128782
Validation Loss: 0.5551038
Epoch: 0452 cost = 0.492135679
Validation Loss: 0.55997854
Epoch: 0453 cost = 0.491163149
Validation Loss: 0.52589226
Epoch: 0454 cost = 0.490192453
Validation Loss: 0.49283588
Epoch: 0455 cost = 0.489334563
Validation Loss: 0.44382635
Epoch: 0456 cost = 0.488357907
Validation Loss: 0.52106476
Epoch: 0457 cost = 0.487405206
Validation Loss: 0.54792154
Epoch: 0458 cost = 0.486450762
Validation Loss: 0.7142284
Epoch: 0459 cost = 0.485612830
Validation Loss: 0.5978283
Epoch: 0460 cost = 0.484651004
Validation Loss: 0.44774902


Epoch: 0591 cost = 0.394529079
Validation Loss: 0.3794118
Epoch: 0592 cost = 0.393967807
Validation Loss: 0.3898354
Epoch: 0593 cost = 0.393614486
Validation Loss: 0.34013072
Epoch: 0594 cost = 0.392985269
Validation Loss: 0.32244313
Epoch: 0595 cost = 0.392546450
Validation Loss: 0.373705
Epoch: 0596 cost = 0.392001003
Validation Loss: 0.35271928
Epoch: 0597 cost = 0.391656061
Validation Loss: 0.37065002
Epoch: 0598 cost = 0.391028206
Validation Loss: 0.40546253
Epoch: 0599 cost = 0.390598938
Validation Loss: 0.37922066
Epoch: 0600 cost = 0.390081197
Validation Loss: 0.36175022
Epoch: 0601 cost = 0.389727627
Validation Loss: 0.32120353
Epoch: 0602 cost = 0.389117787
Validation Loss: 0.35609287
Epoch: 0603 cost = 0.388682693
Validation Loss: 0.32862678
Epoch: 0604 cost = 0.388203487
Validation Loss: 0.3296634
Epoch: 0605 cost = 0.387822886
Validation Loss: 0.42304498
Epoch: 0606 cost = 0.387247235
Validation Loss: 0.43582767
Epoch: 0607 cost = 0.386794562
Validation Loss: 0.40882382
Ep

...

Epoch: 4431 cost = 0.048023425
Validation Loss: 0.05913465
Epoch: 4432 cost = 0.047669146
Validation Loss: 0.058236703
Epoch: 4433 cost = 0.047641225
Validation Loss: 0.05744428
Epoch: 4434 cost = 0.047447079
Validation Loss: 0.057374552
Epoch: 4435 cost = 0.047432022
Validation Loss: 0.057042558
Epoch: 4436 cost = 0.047304250
Validation Loss: 0.0569299
Epoch: 4437 cost = 0.047282364
Validation Loss: 0.056313377
Epoch: 4438 cost = 0.047183220
Validation Loss: 0.056521595
Epoch: 4439 cost = 0.047153649
Validation Loss: 0.056499213
Epoch: 4440 cost = 0.047080281
Validation Loss: 0.057158407
Epoch: 4441 cost = 0.047045728
Validation Loss: 0.056958985
Epoch: 4442 cost = 0.046988508
Validation Loss: 0.05651901
Epoch: 4443 cost = 0.046949421
Validation Loss: 0.056309216
Epoch: 4444 cost = 0.046906381
Validation Loss: 0.056266192
Epoch: 4445 cost = 0.046865676
Validation Loss: 0.055911377
Epoch: 4446 cost = 0.046826612
Validation Loss: 0.055342495
Epoch: 4447 cost = 0.046784689
Validation Los

Epoch: 4577 cost = 0.045730898
Validation Loss: 0.053972736
Epoch: 4578 cost = 0.045729450
Validation Loss: 0.053982306
Epoch: 4579 cost = 0.045727879
Validation Loss: 0.05427057
Epoch: 4580 cost = 0.045726498
Validation Loss: 0.05439432
Epoch: 4581 cost = 0.045725158
Validation Loss: 0.054312143
Epoch: 4582 cost = 0.045723577
Validation Loss: 0.054136828
Epoch: 4583 cost = 0.045722095
Validation Loss: 0.05406376
Epoch: 4584 cost = 0.045720862
Validation Loss: 0.053863995
Epoch: 4585 cost = 0.045719378
Validation Loss: 0.1512945
Epoch: 4586 cost = 0.045717826
Validation Loss: 0.057068296
Epoch: 4587 cost = 0.045716620
Validation Loss: 0.055042382
Epoch: 4588 cost = 0.045715322
Validation Loss: 0.05465217
Epoch: 4589 cost = 0.045713655
Validation Loss: 0.05460198
Epoch: 4590 cost = 0.045712423
Validation Loss: 0.054124814
Epoch: 4591 cost = 0.045711400
Validation Loss: 0.053872988
Epoch: 4592 cost = 0.045709650
Validation Loss: 0.054086093
Epoch: 4593 cost = 0.045708158
Validation Loss:

Epoch: 4715 cost = 0.044833781
Validation Loss: 0.14610447
Epoch: 4716 cost = 0.044695527
Validation Loss: 0.055135053
Epoch: 4717 cost = 0.044744319
Validation Loss: 0.053030677
Epoch: 4718 cost = 0.044812040
Validation Loss: 0.053809687
Epoch: 4719 cost = 0.044669215
Validation Loss: 0.053284753
Epoch: 4720 cost = 0.044715350
Validation Loss: 0.052638218
Epoch: 4721 cost = 0.044777837
Validation Loss: 0.1449744
Epoch: 4722 cost = 0.044629009
Validation Loss: 0.055288583
Epoch: 4723 cost = 0.044666971
Validation Loss: 0.0536053
Epoch: 4724 cost = 0.044715662
Validation Loss: 0.054071512
Epoch: 4725 cost = 0.044550663
Validation Loss: 0.053149465
Epoch: 4726 cost = 0.044564558
Validation Loss: 0.053036675
Epoch: 4727 cost = 0.044578820
Validation Loss: 0.05447282
Epoch: 4728 cost = 0.044376205
Validation Loss: 0.052570056
Epoch: 4729 cost = 0.044352606
Validation Loss: 0.150755
Epoch: 4730 cost = 0.044336693
Validation Loss: 0.05432404
Epoch: 4731 cost = 0.044121594
Validation Loss: 0.

Epoch: 4854 cost = 0.042311691
Validation Loss: 0.049549285
Epoch: 4855 cost = 0.042338622
Validation Loss: 0.04980305
Epoch: 4856 cost = 0.042297585
Validation Loss: 0.049820106
Epoch: 4857 cost = 0.042374890
Validation Loss: 0.049752153
Epoch: 4858 cost = 0.042467929
Validation Loss: 0.05000766
Epoch: 4859 cost = 0.042388381
Validation Loss: 0.048813276
Epoch: 4860 cost = 0.042608291
Validation Loss: 0.052135114
Epoch: 4861 cost = 0.042893461
Validation Loss: 0.050762873
Epoch: 4862 cost = 0.042765683
Validation Loss: 0.050377224
Epoch: 4863 cost = 0.044456525
Validation Loss: 0.054872874
Epoch: 4864 cost = 0.052074730
Validation Loss: 0.083105326
Epoch: 4865 cost = 0.067341461
Validation Loss: 0.08886091
Epoch: 4866 cost = 0.064372708
Validation Loss: 0.075877495
Epoch: 4867 cost = 0.054768405
Validation Loss: 0.060123358
Epoch: 4868 cost = 0.062421512
Validation Loss: 0.07110347
Epoch: 4869 cost = 0.060759551
Validation Loss: 0.12916398
Epoch: 4870 cost = 0.070164463
Validation Los

Epoch: 4997 cost = 0.040995974
Validation Loss: 0.04747931
Epoch: 4998 cost = 0.040996307
Validation Loss: 0.047702014
Epoch: 4999 cost = 0.040992097
Validation Loss: 0.04737636
Epoch: 5000 cost = 0.040992346
Validation Loss: 0.04764105
Optimization Finished!
Test Loss: 0.031278975


In [23]:
import csv
with open(log_files_path + 'Y3layer17-13-9.csv', 'w',newline='') as myfile:
     wr = csv.writer(myfile)
     for i in range(len(togive2)):
        wr.writerow(list(togive2[i][0]))